In [ ]:
from copy import deepcopy
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [ ]:
from flex.data import FlexDataObject, FlexDataset, FlexDatasetConfig, FlexDataDistribution
from flex.pool import FlexPool, FlexModel

In [ ]:
print(tf.__version__)

In [ ]:
print(tfds.__version__)

In [ ]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                    batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [ ]:
print(f"Training entries: {len(train_examples)}, test entries: {len(test_examples)}")

Create the FlexDataObject

In [ ]:
flex_data = FlexDataObject(X_data=train_examples, y_data=train_labels)

In [ ]:
flex_data.validate()

In [ ]:
config = FlexDatasetConfig(seed=0)
config.n_clients = 2
config.replacement = False # ensure that clients do not share any data
config.client_names = ['client1', 'client2']
# config.weights = [0.2] * config.n_clients # each client has only 20% of its assigned class
config.weights = None
flex_dataset = FlexDataDistribution.from_config(cdata=flex_data, config=config)

In [ ]:
# flex_dataset = FlexDataDistribution.iid_distribution(flex_data, n_clients=2)

### Generating the clients and the model to train.

Once we've federated the dataset, we have to create the FlexPool. The FlexPool class simulates a real-time scenario for federated learning, so it is in charge of the communications across the actors. The class FlexPool will assign to each actor a role (client, aggregator, server), so they can communicate during the training phase.

Please, check the notebook about the actors (TODO: Hacer notebook actores y sus relaciones) to know more about the actors and their relationships in FLEXible.

To create a Pool of actors, we need to have a federated dataset, like we've just done, and a model to initialize in the server side, because the server will send the model to the clients so they can train the model. As we have the federated dataset (flex_dataset), we will now create the model.

In this case, we will use a model from the tensorflow hub, so we dont have worry about the preprocessing.

In [ ]:
from flex.pool import FlexPool

In [ ]:
def initialize_server_model(flex_model, *args, **kwargs):
    print("Initializing model server.")
    # model = "https://tfhub.dev/google/nnlm-en-dim50/2" # Not working right now, but it's a lower model.
    model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
    hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam',
                    loss=tf.losses.BinaryCrossentropy(from_logits=True),
                    metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])
    flex_model['model'] = model

In [ ]:
flex_pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=initialize_server_model)

In [ ]:
def deploy_model_to_clients(server_model, clients_model, *args, **kwargs):
    print("Initializing model at client.")
    for client_id in clients_model:
        clients_model[client_id] = deepcopy(server_model)

In [ ]:
clients = flex_pool.clients
server = flex_pool.servers

In [ ]:
server.map(deploy_model_to_clients, clients)

In [ ]:
clients._actors.keys() # Check the clients that will participate in the training of the federated model.

One the model is deployed on the clients, is time to create the training function.

In [ ]:
def train(client_model, data, *args, **kwargs):
    print("Training model at client.")
    model = client_model['model']
    # client_dataset = tf.data.Dataset.from_tensor_slices((data.X_data, data.y_data))
    X_data = data.X_data
    y_data = data.y_data
    history = model.fit(X_data, y_data, epochs=kwargs['epochs'], batch_size=kwargs['batch_size'],
                verbose=1)

In [ ]:
clients.map(train, batch_size=512, epochs=1)

Now that we have trained the model we have to aggregate the weights. To do so, clients will send the weights to the aggregator, and shed will perform the aggregation told. For the tutorial, we will implement the FevAvg aggregation mechanism.

First, we select the aggregator

In [ ]:
aggregator = flex_pool.aggregators

aggregator._models['server_10865309248']:<keras.engine.sequential.Sequential object at 0x107f65340>}
aggregator._models['server_10865309248']['model']:<keras.engine.sequential.Sequential object at 0x107f65340>}


The **map** function from *FlexPool* 

In [ ]:
def collect_weights(client_model, aggregator_model, **kwargs):
    # Here the server and the aggregator are the same, so we need to take the ID from the server
    # to select the model.
    # As the server has a unique ID, we don't know the ID from the server till it's created, so we
    # need to take the ID in this way.
    if 'weights' not in aggregator_model["server"].keys():
        print("Aggregating weights.")
        aggregator_model["server"]['weights'] = []

    aggregator_model["server"]['weights'].append(client_model['model'].get_weights())

In [ ]:
clients.map(collect_weights, aggregator)

**# TODO: Hace falta función de COLECCIÓN de pesos y otra función de AGREGACIÓN de los pesos. Así queda más claro cual es más claro el proceso de colección y cuál es el procedimiento de agregación. Así en el proceso de colección, podemos filtrar por clientes que tengan un modelo inicializado o no.**

In [ ]:
def aggregate_weights(agg_model, *args):
    # agg_model["weights"] = np.mean(np.array(agg_model['weights']), axis=0)
    agg_model["model"].set_weights(np.mean(np.array(agg_model['weights']), axis=0))
    del agg_model["weights"]

In [ ]:
aggregator.map(aggregate_weights)

Now that the aggregator has the aggregated weights, she should send it to the server. To do so, we will use the *map* function to set the new weights to the server model.

In [ ]:
def send_aggregated_weights(aggregator_model, server_model, *args, **kwargs):
    print("Sending aggregated weights to the server.")
    if 'weights' not in aggregator_model.keys():
        raise ValueError('Aggregator should have weights')
    for serv in server_model:
        server_model[serv]['model'].set_weights(aggregator_model['weights'])

In [ ]:
aggregator.map(send_aggregated_weights, server)

Now it's turn from the server to update the weights from the clients models and then evaluate the model.

In [ ]:
def deploy_global_model_to_clients(server_model, clients_models, *args, **kwargs):
    print("Deploying the global model on the clients.")
    aggregated_weights = server_model['model'].get_weights()
    for client_model in clients_models:
        clients_models[client_model]['model'].set_weights(aggregated_weights)

In [ ]:
server.map(deploy_global_model_to_clients, clients)

And now, we can evaluate the model with the test set that we prepared at the begining of the notebook.

In [ ]:
def evaluate_model(model, data, *args, **kwargs):
    model = model['model']
    if data is not None:
        print("Evaluating model at client.")
        results_local = model.evaluate(data.X_data, data.y_data)
        print(f"Results at client on client's data: {results_local}")
    else:
        print("Evaluating model at server")
    results = model.evaluate(kwargs['test_examples'], kwargs['test_labels'])
    print(f"Results on test data: {results}")

In [ ]:
server.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

In [ ]:
clients.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

#### Putting it all together

You just have trained a model for 1 round using FLEXible. Now, you could set up all together in a function and iterate for multiple rounds.

In [ ]:
def train_n_rounds(n_rounds, batch_size, epochs):
    pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=initialize_server_model)
    pool.servers.map(deploy_model_to_clients, pool.clients)
    for i in range(n_rounds):
        print(f"\nRunning round: {i}\n")
        pool.clients.map(train, batch_size=batch_size, epochs=epochs)
        pool.clients.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)
        pool.clients.map(collect_weights, pool.aggregators)
        pool.aggregators.map(aggregate_weights)
        # pool.aggregators.map(send_aggregated_weights, pool.servers) # No hace falta ahora
        pool.servers.map(deploy_global_model_to_clients, pool.clients)
        pool.servers.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

In [ ]:
train_n_rounds(n_rounds=4, batch_size=512, epochs=10)

### END
Congratulations, now you know how to train a model using FLEXible for multiples rounds. Remember that it's important to first deploy/initialize the model on the clients, so you can run the rounds without problem!